In [1]:
import os
import random
import math
import torch
import torch.utils.data as data
import pandas as pd
from PIL import Image
import numpy as np
import cv2
import json

#### 骨头、关键点、角度的转换

In [2]:
def angle2type_V2(alpha,beta):
    if alpha >=60:
        if beta<=55:
            return 'Ia'
        else:
            return 'Ib'
    elif alpha >=50:
        return 'IIa'
    elif alpha >= 43 :
        if beta <= 77:       
            return 'IIc'
        else:
            return 'D'
    else :
        return 'III'

def angle2type_V1(alpha,beta):
    if alpha >=60:
        if beta<=55:
            return 'I'
        else:
            return 'I'
    elif alpha >=50:
        return 'II'
    elif alpha >= 43 :
        if beta <= 77:       
            return 'II'
        else:
            return 'D'
    else :
        return 'III'

def mask2point(mask):
    point_list = np.zeros((6,2))
    # 获取1号点，用2号区域的最左端代替
    if np.sum(mask==2) > 0:
        index = np.argwhere(mask==2)
    else:
        index = np.argwhere(mask==0)
    y = np.argmin(index[:,1])
    point_list[1-1] = index[y]
    # 获取2号点，用1号点平移代替
    point_list[2-1][0] = point_list[1-1][0].item()
    point_list[2-1][1] = point_list[1-1][1].item()+20
    # 获取3号点，用1号区域的最右端代替
    if np.sum(mask==1) > 0:
        index = np.argwhere(mask==1)
    else:
        index = np.argwhere(mask==0)
    y = np.argmax(index[:,1])
    point_list[3-1] = index[y]
    
    # 获取4号点，位于区域3最下端和最右端的平均
    if np.sum(mask==3) > 0:
        index = np.argwhere(mask==3)
    else:
        index = np.argwhere(mask==0)
    y1 = np.argmax(index[:,0])
    y2 = np.argmax(index[:,1])
    point_list[4-1] = np.round(((index[y1]+index[y2])/2))
    '''
    # 获取4号点，位于区域3最右端
    if np.sum(mask==3) > 0:
        index = np.argwhere(mask==3)
    else:
        index = np.argwhere(mask==0)
    y2 = np.argmax(index[:,1])
    point_list[4-1] = index[y2]
    '''
    # 获取5号点，位于区域1最下端
    if np.sum(mask==1) > 0:
        index = np.argwhere(mask==1)
    else:
        index = np.argwhere(mask==0)
    index = np.argwhere(mask==1)
    y = np.argmax(index[:,0])
    point_list[5-1] = index[y]
    # 获取6号点，位于区域4几何中心
    if np.sum(mask==4) > 0:
        index = np.argwhere(mask==4)
    else:
        index = np.argwhere(mask==0)
    p = np.mean(index, axis=0)
    point_list[6-1] = np.round(p)
    return point_list

def point2line(point1,point2):
    # 用极坐标 r,rho 来表示过两点的直线

    # 提取坐标值
    x1, y1 = point1
    x2, y2 = point2

    # 计算极径（r）原点到直线的距离
    A = y2 - y1
    B = x1 - x2
    C = x2 * y1 - x1 * y2
    # 计算原点到直线的距离（点到直线的距离公式）
    r = np.abs(C) / np.sqrt(A**2 + B**2)

    # direction 切点位于y轴的上方还是下方，也就是直线的截距
    direction = (y1*x2-y2*x1)*(x2-x1)
    if direction >=0:
        direction = 1
    else:
        direction = -1

    r*= direction

    # 计算极角（theta）（以弧度为单位）
    theta = np.arctan2(y2 - y1, x2 - x1)+np.pi/2    # 切线和直线垂直
    # +pi/2是把极轴从x轴转为y轴

    # 将极角从弧度转换为度
    theta_degrees = np.degrees(theta)

    return r,theta_degrees  # 其实是极坐标表示的切点位置，从y轴顺时针旋转

def points2angle(points):
    points = points.reshape(3,2,2)
    line_list = [point2line(ps[0], ps[1]) for ps in points ]
    theta_list = [line[1] for line in line_list ]
    alpha,beta = line2angle(*tuple(theta_list))
    return [alpha,beta]

def line2angle(theta1,theta2,theta3):
    # 这个theta是直线的切点的角度，相当于从上y轴开始顺时针
    theta1 = 90
    alpha = theta2-theta1
    beta = theta1-theta3
    return alpha,beta

#### loss 指标

In [9]:
def IOU_metric(pred,label, class_num):
    '''
        pred, label : shape(H,W) , number 0~4
    '''
    IOU_list = []
    for i in range(1,class_num):
        index_pred = (pred==i)
        index_label = (label==i)
        IOU_list.append( (index_pred&index_label).sum() / ((index_pred|index_label).sum()+1e-5) )
    return IOU_list

def DSC_metric(pred,label, class_num):
    '''
        pred, label : shape(H,W) , number 0~4
    '''
    DSC_list = []
    for i in range(1,class_num):
        index_pred = (pred==i)
        index_label = (label==i)
        DSC_list.append( 2*(index_pred&index_label).sum() / (index_pred.sum()+1e-5+index_label.sum()) )
    return DSC_list

#### 文件读取

In [10]:
def read_files(file_list):
    files = []
    for item in file_list:
        image_path, mask_path, point_path = item
        name = os.path.splitext(os.path.basename(mask_path))[0]
        sample = {
            'img': image_path,
            'label': mask_path,
            'keypoint':point_path,
            'name': name
        }
        files.append(sample)
    return files

#### 评估性能

In [11]:
def seg_evalute(name_dir, label_dir, pred_dir):
    result = {
        'IOU':[],
        'DSC':[],
        'alpha':[],
        'beta':[],
        'typeV1':[],
        'typeV2':[]
    }

    name_list = [line.strip() for line in open(name_dir)]
    for name in name_list:
        pred_mask = np.array(Image.open(pred_dir+f'/{name}.png').convert('P'))
        label_mask = np.array(Image.open(label_dir+f'/{name}.png').convert('P'))
        # shape = (H,W) 行,列  当H>W，图片是竖着的长方形
        IOU = IOU_metric(pred_mask, label_mask, class_num=5)
        result['IOU'].append(IOU)

        DSC = DSC_metric(pred_mask, label_mask, class_num=5)
        result['DSC'].append(DSC)

        pred_point = mask2point(pred_mask)
        pred_point = pred_point[:,[1,0]]
        label_point = mask2point(label_mask)
        label_point = label_point[:,[1,0]]

        label_angle = points2angle(label_point)
        pred_angle = points2angle(pred_point)

        alpha_error = abs(pred_angle[0]-label_angle[0])
        beta_error = abs(pred_angle[1]-label_angle[1])
        result['alpha'].append(alpha_error)
        result['beta'].append(beta_error)

        typeV1 = angle2type_V1(*tuple(label_angle)) == angle2type_V1(*tuple(pred_angle))
        typeV2 = angle2type_V2(*tuple(label_angle)) == angle2type_V2(*tuple(pred_angle))
        result['typeV1'].append(typeV1)
        result['typeV2'].append(typeV2)

    for key,value in result.items():
        result[key] = np.array(value)

    return result
    

#### 保存结果

In [12]:
import pandas as pd

def save_result(result,save_dir):
    # 数据
    data = {}
    for i in range(4):
        data[f'IOU 骨头{i+1}']=[round(result['IOU'].mean(0)[i],4)]
    data['IOU 平均']=[round(result['IOU'].mean(),4)]
    for i in range(4):
        data[f'DSC 骨头{i+1}']=[round(result['DSC'].mean(0)[i],4)]
    data['DSC 平均']=[round(result['DSC'].mean(),4)]
    data['alpha']=[round(result['alpha'].mean(),2)]
    data['beta']=[round(result['beta'].mean(),2)]
    data['typeV1']=[round(result['typeV1'].mean(),4)]
    data['typeV2']=[round(result['typeV2'].mean(),4)]

    # 创建DataFrame
    df = pd.DataFrame(data)

    # 保存为Excel文件
    file_path = save_dir+'.xlsx'
    df.to_excel(file_path, index=False)

#### main

In [13]:
log_name_list = ['unet','upper_r50','twins','swin_upernet','setr','segnext-B','deeplabv3','hrnet','mask2former']
for log_name in log_name_list:
    name_dir = '/media/HDD/hln/dataset/HipJoint_mmseg/splits/test.txt'
    label_dir = '/media/HDD/hln/dataset/HipJoint_mmseg/labels'
    pred_dir = f'../work_dirs/{log_name}/pred'
    save_dir = f'../saved/{log_name}'

    result = seg_evalute(name_dir, label_dir, pred_dir)
    save_result(result, save_dir)

    print(f"各个骨头结构IOU{result['IOU'].mean(0)} , 平均IOU{result['IOU'].mean()}")
    print(f"各个骨头结构DSC{result['DSC'].mean(0)} , 平均DSC{result['DSC'].mean()}")
    print(f"alpha 角度误差 {result['alpha'].mean()}, beta 角度误差 {result['beta'].mean()}" )
    print(f"typeV1疾病准确率 {result['typeV1'].mean()}, typeV2疾病准确 {result['typeV2'].mean()}" )

各个骨头结构IOU[0.75286922 0.65907641 0.68782659 0.51736375] , 平均IOU0.6542839927007845
各个骨头结构DSC[0.85569319 0.78790815 0.80158121 0.66048621] , 平均DSC0.7764171916709588
alpha 角度误差 6.961525280362061, beta 角度误差 3.9799574181370465
typeV1疾病准确率 0.8089171974522293, typeV2疾病准确 0.7133757961783439
各个骨头结构IOU[0.69580205 0.61088572 0.65606685 0.48589065] , 平均IOU0.6121613163982725
各个骨头结构DSC[0.81318455 0.74786179 0.78346542 0.62734511] , 平均DSC0.7429642156697546
alpha 角度误差 3.5874731610098354, beta 角度误差 4.877796896210727
typeV1疾病准确率 0.8535031847133758, typeV2疾病准确 0.732484076433121
各个骨头结构IOU[0.7007907  0.64247871 0.67797936 0.49596392] , 平均IOU0.6293031719606652
各个骨头结构DSC[0.81617566 0.77453853 0.79958942 0.63767932] , 平均DSC0.7569957312924762
alpha 角度误差 3.500032354957647, beta 角度误差 4.336021750611191
typeV1疾病准确率 0.8280254777070064, typeV2疾病准确 0.7452229299363057
各个骨头结构IOU[0.67889153 0.61610918 0.65031943 0.47504521] , 平均IOU0.6050913399547292
各个骨头结构DSC[0.80000437 0.7516194  0.7752504  0.61813358] , 平均DSC0.73625193

In [1]:
round(3.14155,4)

3.1416

In [1]:
def iou(x):
    return x/(2-x) 

In [14]:
iou(0.8002)

0.6669444907484581

In [16]:
import numpy as np
np.mean([749,668,739,563])

679.75